In [125]:
import sys        
import traceback
import random
import math
import time
import _thread
import threading
import pdb
from pdb import set_trace as bp
from scipy.integrate import *

import numpy as np
%reload_ext Cython

In [126]:
class Position:
    _x = 0
    _y = 0
    _z = 0
    def __init__ (self,x,y,z):
        self._x = x
        self._y = y
        self._z = z
    def print_info(self):
        print ('_x:',self._x, '_y:',self._y, '_z:',self._z)
        
class Speed:
    _u = 0
    _v = 0
    _w = 0
    def __init__ (self,u,v,w):
        self._u = u
        self._v = v
        self._w = w
    def print_info(self):
        print ('_u:',self._u, '_v:',self._v, '_w:',self._w)

class Item(Position, Speed):
    _m = 0
    _color = ''
    _lifetime = 0    
    def __init__(self,x,y,z,u,v,w,m,color,lifetime):
        Position.__init__(self, x, y, z)
        Speed.__init__(self, u, v, w)
        self._m = m
        self._color = color
        self._lifetime = lifetime
    def print_info(self):
        print ('_x:',self._x, '_y:', self._y, '_z:', self._z,'_u:', self._u, '_v:', self._v, '_w:', self._w, '_m:', self._m, '_color:', 

self._color, '_lifetime', self._lifetime)
        
class Emitter(Position,Speed):
    def __init__(self,x,y,z,u,v,w):
        Position.__init__(self, x, y, z)
        Speed.__init__(self, u, v, w)
    def print_info(self):
        print ('_x:',self._x, '_y:', self._y, '_z:', self._z,'_u:', self._u, '_v:', self._v, '_w:', self._w)
        
class myThreadPosition(threading.Thread): 

    def __init__(self, _id, _pos_prev, _speed_prev, _a_prev, _delta_t, _num): 
        threading.Thread.__init__(self) 
        self.threadID = _id 
        self.pos_prev = _pos_prev 
        self.speed_prev = _speed_prev 
        self.a_prev = _a_prev 
        self.delta_t = _delta_t
        self.num = _num
        self.value = 0 
    
    def run(self): 
        self.value = self.pos_prev[self.num] + self.speed_prev[self.num]*self.delta_t + 1/2*self.a_prev[self.num]*(self.delta_t**2) 

    def Value(self): 
        return self.value
    
class myThreadSpeed(threading.Thread): 

    def __init__(self, _id, _speed_prev, _a_prev, _a_n, _delta_t, _num): 
        threading.Thread.__init__(self) 
        self.threadID = _id 
        self.speed_prev = _speed_prev 
        self.a_prev = _a_prev 
        self.a_n = _a_n 
        self.delta_t = _delta_t
        self.num = _num
        self.value = 0 
    
    def run(self): 
        self.value = self.speed_prev[self.num] + 1/2*(self.a_prev[self.num]+self.a_n[self.num])*self.delta_t

    def Value(self): 
        return self.value

def items_generate(N): 
    global s1
    color_list = ["green", "red", "blue", "yellow", "orange"]
    for i in range(N):
        x_item = random.uniform(-1, 1)
        y_item = random.uniform(-1, 1)
        z_item = random.uniform(-1, 1)
        u_item = random.uniform(-1, 1)
        v_item = random.uniform(-1, 1)
        w_item = random.uniform(-1, 1)
        m_item = random.uniform(100000, 100000000)
        color_item = random.choice(color_list)
        time_item = random.uniform(1000000, 5000000000)
        s1.append(Item(x_item,y_item,z_item,u_item,v_item,w_item,m_item,color_item,time_item))
#        for i in range(len(s1)):
#            print(s1[i].print_info())    
    return

In [127]:
def calculate_Verlet(s1, num_of_iterations, delta_t):
    print(len(s1))
    list_of_timelife = []
    binary_time = []
    result_list = []
    time_average = [] 
    mass_coef = 5.9726*(10)**24
    current_time = 0
    count = 1
    temp_bool = 0                                         #item_exist
    for l in range(len(s1)):                            
        s1[l]._m = s1[l]._m * mass_coef
    for k in range(len(s1)):
        list_of_timelife.append(s1[k]._lifetime)
        binary_time.append(1)
    for l in range(len(s1)):                               #initialization of 2-D list
        x = []
        result_list.append(x)
        result_list[l].append(s1[l])
    start_time = 0
    while (count <= num_of_iterations and temp_bool == 0): 
        start_time = time.time()
        for k in range(len(binary_time)):                  #dead or not dead
            if (list_of_timelife[k] < current_time):
                binary_time[k] = 0 
        if (all(item == 0 for item in binary_time) == True):
            temp_bool = 1
            print("end while")
            break
        x_prev = []
        y_prev = []
        z_prev = []
        u_prev = []
        v_prev = []
        w_prev = []
        m_prev = []
        color_prev = []
        lifetime_prev = []
        for m in range(len(result_list)):                  #remember previous coordinates for each item//
            index = len(result_list[m]) - 1                #fill in lists
            x_prev.append(result_list[m][index]._x)
            y_prev.append(result_list[m][index]._y)
            z_prev.append(result_list[m][index]._z)
            u_prev.append(result_list[m][index]._u)
            v_prev.append(result_list[m][index]._v)
            w_prev.append(result_list[m][index]._w)
            m_prev.append(result_list[m][index]._m)
            color_prev.append(result_list[m][index]._color)
            lifetime_prev.append(result_list[m][index]._lifetime)
        a1_prev = []
        a2_prev = []
        a3_prev = [] 
        for z in range(len(result_list)):                   #calculate previous acceleration for each item
            a1_prev.append(0)
            a2_prev.append(0)
            a3_prev.append(0)
            for n in range(len(result_list)):               #sum in acceleration's calculation
                if (n != z):
                    const_absolute = math.sqrt((x_prev[n] - x_prev[z])**2 + (y_prev[n] - y_prev[z])**2 + (z_prev[n] - z_prev[z])**2)
                    const = 6.67408*(10**(-20))*m_prev[n]
                    if const_absolute < 1:
                        const_absolute = 1
                    a1_prev[z] += const*(x_prev[n] - x_prev[z])/(const_absolute**3) 
                    a2_prev[z] += const*(y_prev[n] - y_prev[z])/(const_absolute**3)
                    a3_prev[z] += const*(z_prev[n] - z_prev[z])/(const_absolute**3)
        x_n = []
        y_n = []
        z_n = []
        for i in range(len(result_list)):                     #next coordinates
            x_n.append(x_prev[i] + u_prev[i]*delta_t + 1/2*a1_prev[i]*(delta_t**2))
            y_n.append(y_prev[i] + v_prev[i]*delta_t + 1/2*a2_prev[i]*(delta_t**2))
            z_n.append(z_prev[i] + w_prev[i]*delta_t + 1/2*a3_prev[i]*(delta_t**2))
        a1_n = []
        a2_n = []
        a3_n = []
        for z in range(len(result_list)):                     #calculate next acceleration for each item
            a1_n.append(0)
            a2_n.append(0)
            a3_n.append(0)
            for n in range(len(result_list)):                 #sum in acceleration's calculation
                if (n != z):
                    const_absolute = math.sqrt((x_n[n] - x_n[z])**2 + (y_n[n] - y_n[z])**2 + (z_n[n] - z_n[z])**2)
                    const = 6.67408*(10**(-20))*m_prev[n]
                    if const_absolute < 1:
                        const_absolute = 1
                    a1_n[z] += const*(x_n[n] - x_n[z])/(const_absolute**3) 
                    a2_n[z] += const*(y_n[n] - y_n[z])/(const_absolute**3)
                    a3_n[z] += const*(z_n[n] - z_n[z])/(const_absolute**3)
        u_n = []
        v_n = []
        w_n = []
        for i in range(len(result_list)):
            u_n.append(u_prev[i] + 1/2*(a1_prev[i]+a1_n[i])*delta_t)
            v_n.append(v_prev[i] + 1/2*(a2_prev[i]+a2_n[i])*delta_t)
            w_n.append(w_prev[i] + 1/2*(a3_prev[i]+a3_n[i])*delta_t)
            if (binary_time[i] != 0):
                result_list[i].append(Item(x_n[i],y_n[i],z_n[i],u_n[i],v_n[i],w_n[i],m_prev[i],color_prev[i],lifetime_prev[i]))
        current_time += delta_t
        count += 1
        time_average.append(time.time()-start_time)  
    return sum(time_average) / float(len(time_average))

In [128]:
def Thread_Verlet(s1, num_of_iterations, delta_t):
    list_of_timelife = []
    binary_time = []
    result_list = []
    time_average = [] 
    mass_coef = 5.9726*(10)**24
    current_time = 0
    count = 1
    temp_bool = 0                                         #item_exist
    for l in range(len(s1)):                            
        s1[l]._m = s1[l]._m * mass_coef
    for k in range(len(s1)):
        list_of_timelife.append(s1[k]._lifetime)
        binary_time.append(1)
    for l in range(len(s1)):                               #initialization of 2-D list
        x = []
        result_list.append(x)
        result_list[l].append(s1[l])
    start_time = 0
    while (count <= num_of_iterations and temp_bool == 0): 
        start_time = time.time()
        for k in range(len(binary_time)):                  #dead or not dead
            if (list_of_timelife[k] < current_time):
                binary_time[k] = 0 
        if (all(item == 0 for item in binary_time) == True):
            temp_bool = 1
            print("end while")
            break
        x_prev = []
        y_prev = []
        z_prev = []
        u_prev = []
        v_prev = []
        w_prev = []
        m_prev = []
        color_prev = []
        lifetime_prev = []
        for m in range(len(result_list)):                  #remember previous coordinates for each item//
            index = len(result_list[m]) - 1                #fill in lists
            x_prev.append(result_list[m][index]._x)
            y_prev.append(result_list[m][index]._y)
            z_prev.append(result_list[m][index]._z)
            u_prev.append(result_list[m][index]._u)
            v_prev.append(result_list[m][index]._v)
            w_prev.append(result_list[m][index]._w)
            m_prev.append(result_list[m][index]._m)
            color_prev.append(result_list[m][index]._color)
            lifetime_prev.append(result_list[m][index]._lifetime)
        a1_prev = []
        a2_prev = []
        a3_prev = [] 
        for z in range(len(result_list)):                   #calculate previous acceleration for each item
            a1_prev.append(0)
            a2_prev.append(0)
            a3_prev.append(0)
            for n in range(len(result_list)):               #sum in acceleration's calculation
                if (n != z):
                    const_absolute = math.sqrt((x_prev[n] - x_prev[z])**2 + (y_prev[n] - y_prev[z])**2 + (z_prev[n] - z_prev[z])**2)
                    const = 6.67408*(10**(-20))*m_prev[n]
                    if const_absolute < 1:
                        const_absolute = 1
                    a1_prev[z] += const*(x_prev[n] - x_prev[z])/(const_absolute**3) 
                    a2_prev[z] += const*(y_prev[n] - y_prev[z])/(const_absolute**3)
                    a3_prev[z] += const*(z_prev[n] - z_prev[z])/(const_absolute**3)
        x_n = []
        y_n = []
        z_n = []
        for i in range(len(result_list)):                     #next coordinates
                thread1 = myThreadPosition(1, x_prev, u_prev, a1_prev, delta_t, i)
                thread2 = myThreadPosition(2, y_prev, v_prev, a2_prev, delta_t, i)
                thread3 = myThreadPosition(3, z_prev, w_prev, a3_prev, delta_t, i)
                thread1.start()
                thread2.start()
                thread3.start()
                x_n.append(thread1.Value())
                y_n.append(thread2.Value())
                z_n.append(thread3.Value())
                thread1.join()
                thread2.join()
                thread3.join()
        a1_n = []
        a2_n = []
        a3_n = []
        for z in range(len(result_list)):                     #calculate next acceleration for each item
            a1_n.append(0)
            a2_n.append(0)
            a3_n.append(0)
            for n in range(len(result_list)):                 #sum in acceleration's calculation
                if (n != z):
                    const_absolute = math.sqrt((x_n[n] - x_n[z])**2 + (y_n[n] - y_n[z])**2 + (z_n[n] - z_n[z])**2)
                    const = 6.67408*(10**(-20))*m_prev[n]
                    if const_absolute < 1:
                        const_absolute = 1
                    a1_n[z] += const*(x_n[n] - x_n[z])/(const_absolute**3) 
                    a2_n[z] += const*(y_n[n] - y_n[z])/(const_absolute**3)
                    a3_n[z] += const*(z_n[n] - z_n[z])/(const_absolute**3)
        u_n = []
        v_n = []
        w_n = []
        for i in range(len(result_list)):
            thread4 = myThreadPosition(1, u_prev, a1_prev, a1_n, delta_t, i)
            thread5 = myThreadPosition(2, v_prev, a2_prev, a2_n, delta_t, i)
            thread6 = myThreadPosition(3, w_prev, a3_prev, a3_n, delta_t, i)
            thread4.start()
            thread5.start()
            thread6.start()
            u_n.append(thread4.Value())
            v_n.append(thread5.Value())
            w_n.append(thread6.Value())
            thread4.join()
            thread5.join()
            thread6.join()
            if (binary_time[i] != 0):
                result_list[i].append(Item(x_n[i],y_n[i],z_n[i],u_n[i],v_n[i],w_n[i],m_prev[i],color_prev[i],lifetime_prev[i]))
        current_time += delta_t
        count += 1
        time_average.append(time.time()-start_time)  
    return sum(time_average) / float(len(time_average))

In [129]:
%%cython

class Position:
    _x = 0
    _y = 0
    _z = 0
    def __init__ (self,x,y,z):
        self._x = x
        self._y = y
        self._z = z
    def print_info(self):
        print ('_x:',self._x, '_y:',self._y, '_z:',self._z)
        
class Speed:
    _u = 0
    _v = 0
    _w = 0
    def __init__ (self,u,v,w):
        self._u = u
        self._v = v
        self._w = w
    def print_info(self):
        print ('_u:',self._u, '_v:',self._v, '_w:',self._w)

class Item(Position, Speed):
    _m = 0
    _color = ''
    _lifetime = 0    
    def __init__(self,x,y,z,u,v,w,m,color,lifetime):
        Position.__init__(self, x, y, z)
        Speed.__init__(self, u, v, w)
        self._m = m
        self._color = color
        self._lifetime = lifetime
    def print_info(self):
        print ('_x:',self._x, '_y:', self._y, '_z:', self._z,'_u:', self._u, '_v:', self._v, '_w:', self._w, '_m:', self._m, '_color:', 
self._color, '_lifetime', self._lifetime)

def Cython_Verlet_calculate(s1, result_list, binary_time, list_of_timelife, current_time, count, delta_t):
    return cython_calculate_fun(s1, result_list, binary_time, list_of_timelife, current_time, count, delta_t)

def cython_calculate_fun(s1, result_list, binary_time, list_of_timelife, current_time, count, delta_t):
    for k in range(len(binary_time)):                  #dead or not dead
        if (list_of_timelife[k] < current_time):
            binary_time[k] = 0 
    if (all(item == 0 for item in binary_time) == True):
        temp_bool = 1
        print("end while")
    x_prev = []
    y_prev = []
    z_prev = []
    u_prev = []
    v_prev = []
    w_prev = []
    m_prev = []
    color_prev = []
    lifetime_prev = []
    for m in range(len(result_list)):                  #remember previous coordinates for each item//
        index = len(result_list[m]) - 1                #fill in lists
        x_prev.append(result_list[m][index]._x)
        y_prev.append(result_list[m][index]._y)
        z_prev.append(result_list[m][index]._z)
        u_prev.append(result_list[m][index]._u)
        v_prev.append(result_list[m][index]._v)
        w_prev.append(result_list[m][index]._w)
        m_prev.append(result_list[m][index]._m)
        color_prev.append(result_list[m][index]._color)
        lifetime_prev.append(result_list[m][index]._lifetime)
    a1_prev = []
    a2_prev = []
    a3_prev = [] 
    for z in range(len(result_list)):                   #calculate previous acceleration for each item
        a1_prev.append(0)
        a2_prev.append(0)
        a3_prev.append(0)
        for n in range(len(result_list)):               #sum in acceleration's calculation
            if (n != z):
                const_absolute = ((x_prev[n] - x_prev[z])**2 + (y_prev[n] - y_prev[z])**2 + (z_prev[n] - z_prev[z])**2)**(0.5)
                const = 6.67408*(10**(-20))*m_prev[n]
                if const_absolute < 1:
                    const_absolute = 1
                a1_prev[z] += const*(x_prev[n] - x_prev[z])/(const_absolute**3) 
                a2_prev[z] += const*(y_prev[n] - y_prev[z])/(const_absolute**3)
                a3_prev[z] += const*(z_prev[n] - z_prev[z])/(const_absolute**3)
    x_n = []
    y_n = []
    z_n = []
    for i in range(len(result_list)):                     #next coordinates
        x_n.append(x_prev[i] + u_prev[i]*delta_t + 1/2*a1_prev[i]*(delta_t**2))
        y_n.append(y_prev[i] + v_prev[i]*delta_t + 1/2*a2_prev[i]*(delta_t**2))
        z_n.append(z_prev[i] + w_prev[i]*delta_t + 1/2*a3_prev[i]*(delta_t**2))
    a1_n = []
    a2_n = []
    a3_n = []
    for z in range(len(result_list)):                     #calculate next acceleration for each item
        a1_n.append(0)
        a2_n.append(0)
        a3_n.append(0)
        for n in range(len(result_list)):                 #sum in acceleration's calculation
            if (n != z):
                const_absolute = ((x_n[n] - x_n[z])**2 + (y_n[n] - y_n[z])**2 + (z_n[n] - z_n[z])**2)**(0.5)
                const = 6.67408*(10**(-20))*m_prev[n]
                if const_absolute < 1:
                    const_absolute = 1
                a1_n[z] += const*(x_n[n] - x_n[z])/(const_absolute**3) 
                a2_n[z] += const*(y_n[n] - y_n[z])/(const_absolute**3)
                a3_n[z] += const*(z_n[n] - z_n[z])/(const_absolute**3)
    u_n = []
    v_n = []
    w_n = []
    for i in range(len(result_list)):
        u_n.append(u_prev[i] + 1/2*(a1_prev[i]+a1_n[i])*delta_t)
        v_n.append(v_prev[i] + 1/2*(a2_prev[i]+a2_n[i])*delta_t)
        w_n.append(w_prev[i] + 1/2*(a3_prev[i]+a3_n[i])*delta_t)
        if (binary_time[i] != 0):
            result_list[i].append(Item(x_n[i],y_n[i],z_n[i],u_n[i],v_n[i],w_n[i],m_prev[i],color_prev[i],lifetime_prev[i]))
    return result_list, binary_time, list_of_timelife

In [130]:
def calculate_Cython(s1, num_of_iterations, delta_t):
    global list_of_timelife
    list_of_timelife = []
    global binary_time 
    binary_time = []
    global result_list 
    result_list = []
    global time_average 
    time_average = [] 
    global temp_bool 
    temp_bool = 0 
    mass_coef = 5.9726*(10)**24
    current_time = 0
    count = 1                                        #item_exist
    for l in range(len(s1)):                            
        s1[l]._m = s1[l]._m * mass_coef
    for k in range(len(s1)):
        list_of_timelife.append(s1[k]._lifetime)
        binary_time.append(1)
    for l in range(len(s1)):                               #initialization of 2-D list
        x = []
        result_list.append(x)
        result_list[l].append(s1[l])
    start_time = 0
    while (count <= num_of_iterations and temp_bool == 0): 
        start_time = time.time()
        result_list, binary_time, list_of_timelife = Cython_Verlet_calculate(s1[:], result_list, binary_time, list_of_timelife, current_time, count, delta_t)
        current_time += delta_t
        count += 1 
        time_average.append(time.time()-start_time) 
    return sum(time_average) / float(len(time_average))
    

In [131]:
if __name__ == '__main__':
    s1 = []
    items_generate(100)
    print("Verlet: ", calculate_Verlet(s1[:], 5, 100000))
    print("Cython_Verlet: ", calculate_Cython(s1[:], 5, 100000))
    print("Thread_Verlet: ", Thread_Verlet(s1[:], 5, 100000))
    s1 = []
    items_generate(200)
    print("Verlet: ", calculate_Verlet(s1[:], 5, 100000))
    print("Cython_Verlet: ", calculate_Cython(s1[:], 5, 100000))
    print("Thread_Verlet: ", Thread_Verlet(s1[:], 5, 100000))
    s1 = []
    items_generate(400)
    print("Verlet: ", calculate_Verlet(s1[:], 5, 100000))
    print("Cython_Verlet: ", calculate_Cython(s1[:], 5, 100000))
    print("Thread_Verlet: ", Thread_Verlet(s1[:], 5, 100000))
    s1 = []
    items_generate(1000)
    print("Verlet: ", calculate_Verlet(s1[:], 5, 100000))
    print("Cython_Verlet: ", calculate_Cython(s1[:], 5, 100000))
    print("Thread_Verlet: ", Thread_Verlet(s1[:], 5, 100000))

100
Verlet:  0.05804462432861328
Cython_Verlet:  0.03907170295715332
Thread_Verlet:  0.5053293704986572
200
Verlet:  0.2191993236541748
Cython_Verlet:  0.16007413864135742
Thread_Verlet:  1.1046483993530274
400
Verlet:  0.865789794921875
Cython_Verlet:  0.6313363552093506
Thread_Verlet:  2.6301721572875976
1000
Verlet:  5.570112514495849
Cython_Verlet:  4.2020491600036625
Thread_Verlet:  10.524673748016358
